In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imb_Pipeline
import functions as F

warnings.filterwarnings(action='ignore')

## Load dataset

In [ ]:
aave_loan_balance = pd.read_csv('./data/aave_loan_balance.csv')
application_previous = pd.read_csv('./data/application_previous.csv')
defi_credits = pd.read_csv('./data/defi_credits.csv')
defi_credits_balance = pd.read_csv('./data/defi_credits_balance.csv')
loan_application = pd.read_csv('./data/loan_application.csv')
payments_history = pd.read_csv('./data/payments_history.csv')

In [ ]:
lst = [aave_loan_balance, application_previous, defi_credits, defi_credits_balance, loan_application, payments_history]

## EDA

#### primary_key

In [ ]:
for i, data in enumerate(lst):
    print(f'{i}th table', end='\t')
    F.has_pk(data)

#### null

In [ ]:
fig, axes = plt.subplots(1, 6, figsize=(15, 5))

for i, data in enumerate(lst):
    sns.histplot(F.check_null_status(data), ax=axes[i], stat='probability')

## Data preparation

#### loan_application (1/6)

In [ ]:
loan_application.set_index('application_id_current', inplace=True)

In [ ]:
prep_loan_application = loan_application.copy()
categorical_features = F.search_categoric(loan_application)

for col in categorical_features:
    prep_loan_application[col] = prep_loan_application[col].astype('object')

#### defi_credits (2/6) & defi_credits_balance (3/6)

In [ ]:
temp = pd.pivot_table(data=defi_credits_balance,
                      index='defi_id',
                      columns='months_balance',
                      values='status')

In [ ]:
prep_defi_credits_balance = temp.apply(np.nanmean, axis=1)
prep_defi_credits_balance.name = 'mean_balance'

In [ ]:
defi_merged = pd.merge(defi_credits, 
                       prep_defi_credits_balance, 
                       how='left', 
                       left_on='defi_id', 
                       right_index=True)

In [ ]:
prep_defi = F.aggregate(defi_merged, 
                        primary_key=['application_id_current'],
                        drop_cols=['defi_id'])

#### aave_loan_balance (4/6)

In [ ]:
prep_aave_loan_balance = F.aggregate(aave_loan_balance,
                                     primary_key=['application_id_current'],
                                     drop_cols=['application_id_previous'])

#### application_previous (5/6)

In [ ]:
application_previous = pd.read_csv('./data/application_previous.csv')

In [ ]:
categorical_features = F.search_categoric(application_previous, cardinality_limit=10)

for col in categorical_features:
    application_previous[col] = application_previous[col].astype('object')

In [ ]:
prep_application_previous = F.aggregate(application_previous,
                                        primary_key=['application_id_current'],
                                        drop_cols=['application_id_previous'])

#### payments_history (6/6)

In [ ]:
payments_history['days_diff'] = payments_history['days_entry_payment'] - payments_history['days_instalment']

In [ ]:
prep_payments_history = F.aggregate(payments_history,
                                    primary_key=['application_id_current'],
                                    drop_cols=['application_id_previous'])

## Merge datasets

In [ ]:
preprocessed = [prep_loan_application, prep_aave_loan_balance, prep_application_previous, prep_defi, prep_payments_history]

for i, table in enumerate(preprocessed):
    print(f'{i} th table index : {table.index.name}')
    if i == 0:
        merged = prep_loan_application.copy()
    else:
        merged = pd.merge(merged, table, how='left', left_index=True, right_index=True)

## Split

In [ ]:
cond = merged.type == 1
train_val = merged.loc[cond]
test = merged.loc[~cond]

X = train_val.drop('target', axis=1)
y = train_val.target.astype(str)

## Define a pipeline

In [ ]:
numeric_transformer = MinMaxScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, selector(dtype_exclude="object")),
        ('cat', categorical_transformer, selector(dtype_include="object"))
    ])

In [ ]:
model = lgb.LGBMClassifier(n_estimators=100)

In [ ]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model)])

In [ ]:
params = {'classifier__learning_rate': [0.01, 0.05, 0.1],
          'classifier__num_leaves': [3, 6, 9],
          'classifier__reg_alpha': [1e-1, 1, 10]
         }

## Hyperparameter tuning

In [ ]:
tuner = RandomizedSearchCV(
    estimator=clf,
    param_distributions=params,
    n_iter=3,
    scoring='roc_auc',
    cv=3,
    refit=True,
    verbose=True,
    n_jobs=-1
)

In [ ]:
tuner.fit(X, y)

In [ ]:
print(f'train auroc: {roc_auc_score(y_train, tuner.predict_proba(X_train)[:, 1]):.4f}')
print(f'val auroc: {roc_auc_score(y_val, tuner.predict_proba(X_val)[:, 1]):.4f}')

## Prediction

In [ ]:
result = pd.Series(
    tuner.predict_proba(test.drop('target', axis=1))[:, 1],
    name='target',
    index=test.index)

In [ ]:
result.to_csv('./Result.csv')